In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

2025-04-15 19:38:15.161146: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 19:38:15.161995: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 19:38:15.165310: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 19:38:15.173066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 19:38:15.188606: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [2]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [3]:
# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_imdb.h5')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
model.get_weights()

[array([[ 1.5024228e-03,  2.5260368e-02, -2.6603080e-02, ...,
         -1.0215807e-02,  8.1480462e-03, -4.8987914e-02],
        [ 1.4427090e-02, -8.3803687e-05,  6.5948693e-03, ...,
          1.1196666e-02, -2.0340223e-02, -5.6525774e-02],
        [ 5.9529260e-02,  7.6480679e-02,  3.4913268e-02, ...,
          2.2568321e-02,  2.9416138e-02,  5.6720201e-02],
        ...,
        [-7.9326943e-02, -5.1991869e-02,  4.5792874e-02, ...,
         -2.0453852e-02,  2.2810886e-02,  1.7712349e-02],
        [ 6.6819508e-04, -6.8679377e-03, -3.8095482e-02, ...,
          3.7731469e-02,  3.0752717e-02, -2.9063795e-02],
        [-6.0768384e-02, -9.5799090e-03,  2.3005376e-02, ...,
          5.1136516e-02, -9.5610015e-02,  4.0149171e-02]], dtype=float32),
 array([[ 0.18056491, -0.03617371, -0.06325124, ...,  0.03957566,
         -0.08670758, -0.14506213],
        [ 0.005446  , -0.12953772, -0.00042415, ...,  0.07935873,
         -0.12150659,  0.02065097],
        [ 0.08124612,  0.18469054,  0.04142294

In [5]:
# Step 2: Helper Functions
# Function to decode reviews
''''
Input

Function takes encoded_review → a list of integers (word indices).

Index Adjustment

Subtract 3 from each index because:

0 = padding

1 = start token

2 = unknown token

Real word indices start from 3.

Lookup in Word Index

Use reverse_word_index.get(i - 3, '?') to get the word:

If the index exists → get the word.

If not → put '?' (unknown word placeholder).

Join Words into Sentence

Use ' '.join([...]) to combine the words into a sentence.

Final output: readable sentence from numbers.
'''
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [6]:
### Prediction  function

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]



In [7]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."
#example_review = "This movie was not fantastic! The acticg was not so great."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.5865349769592285
